# saveとload

In [1]:
import torch
import torch.nn as nn

## save

https://pytorch.org/docs/stable/generated/torch.save.html

テンソルやモデルの保存。拡張子には`.pt`や`.pth`が基本使われる。

どちらを使うべき、みたいな話は多分なくて、本当にどっちでも良さそう。公式にも

> A common PyTorch convention is to save models using either a `.pt` or `.pth` file extension.

って書いてある。ただ`.pt`はzopeというwebフレームワークで用いるテンプレートファイルにも使われるようで、一応重複を避けようという気持ちで私は`.pth`を使っている。`.pt`にするとvscodeでテキストファイルのアイコンなんかが出て気持ち悪かったりもする。

In [2]:
x = torch.tensor([1, 2, 3])
torch.save(x, "data.pth")

In [3]:
model = nn.Linear(3, 1)
torch.save(model, "model.pth") # この使い方はしない方がいい（詳細は後で）

別にテンソルやモデルでなくてもいい。任意のオブジェクトを保存することができる。pickleと同じ。

In [4]:
class MyObject:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

    def __repr__(self):
        return f"MyObject(a={self.a}, b={self.b}, c={self.c})"

obj = MyObject(1, 2, 3)
torch.save(obj, "obj.pth")

ただ、この使い方もしない方がいい。普通にpickle使おう。

## load

https://pytorch.org/docs/stable/generated/torch.load.html

saveしたファイルを読み込む。

In [5]:
data = torch.load("data.pth")
data

/tmp/ipykernel_1614364/1216731273.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("data.pth")


tensor([1, 2, 3])

In [6]:
model = torch.load("model.pth")
model

/tmp/ipykernel_1614364/3675544202.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("model.pth")


Linear(in_features=3, out_features=1, bias=True)

In [7]:
obj = torch.load("obj.pth")
obj

/tmp/ipykernel_1614364/2542391371.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  obj = torch.load("obj.pth")


MyObject(a=1, b=2, c=3)

ファイルは全て読み込めたけど、なんか警告出たな。

「今は`weights_only=False`がデフォルトだけど、近いうちに`True`にするから注意してや」って言ってるね。

In [8]:
"ちなみに今のバージョンは" + torch.__version__

'ちなみに今のバージョンは2.4.0+cu121'

`weights_only=True`は、読み込めるオブジェクトをテンソルと辞書（+α）だけにするということ。

モデルのsaveには二つのやり方があって、一つはさっきやったようにモデルをそのまま保存するやり方。もう一つはモデルの重みだけを保存するやり方で、`model.state_dict()`を使う（詳細は後程）。こうすると重みだけが辞書として保存される。

`weights_only=True`だと前者のやり方が使えなくなる。これをデフォルトにすると言っているので、今後は前者を使いたい場合に`weights_only=False`を明示的に指定する必要がある。まあ余程のことがない限り後者を使った方がいいと思うけど。わざわざデフォルト値を変えるということは、そこに問題があったということなので、大人しく従った方がいい。実際にモデルのsaveには`state_dict()`を使うことが推奨されている。

> Instead of saving a module directly, for compatibility reasons it is recommended to instead save only its state dict.

https://pytorch.org/docs/stable/notes/serialization.html#saving-and-loading-torch-nn-modules